In [1]:
#import necessary libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
#data preprocessing and clustering
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import plotly.graph_objects as go
import plotly.express as px
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [2]:
#read in csv file
customer = pd.read_csv("Orders.csv")
customer.head()

,product_title,product_type,variant_title,variant_sku,variant_id,customer_id,order_id,day,net_quantity,gross_sales,discounts,returns,net_sales,taxes,total_sales,returned_item_quantity,ordered_item_quantity
0,DPR,DPR,100,AD-982-708-895-F-6C894FB,52039657,1312378,83290718932496,04/12/2018,2,200.0,-200.00,0.00,0.0,0.0,0.0,0,2
1,RJF,Product P,28 / A / MTM,83-490-E49-8C8-8-3B100BC,56914686,3715657,36253792848113,01/04/2019,2,190.0,-190.00,0.00,0.0,0.0,0.0,0,2
2,CLH,Product B,32 / B / FtO,68-ECA-BC7-3B2-A-E73DE1B,24064862,9533448,73094559597229,05/11/2018,0,164.8,-156.56,-8.24,0.0,0.0,0.0,-2,2
3,NMA,Product F,40 / B / FtO,6C-1F1-226-1B3-2-3542B41,43823868,4121004,53616575668264,19/02/2019,1,119.0,-119.00,0.00,0.0,0.0,0.0,0,1
4,NMA,Product F,40 / B / FtO,6C-1F1-226-1B3-2-3542B41,43823868,4121004,29263220319421,19/02/2019,1,119.0,-119.00,0.00,0.0,0.0,0.0,0,1


In [3]:
#analyse the state of the file
customer.shape

(70052, 17)

In [4]:
customer.describe()

,variant_id,customer_id,order_id,net_quantity,gross_sales,discounts,returns,net_sales,taxes,total_sales,returned_item_quantity,ordered_item_quantity
count,7.005200e+04,7.005200e+04,7.005200e+04,70052.000000,70052.000000,70052.000000,70052.000000,70052.000000,70052.000000,70052.000000,70052.000000,70052.000000
mean,2.442320e+11,6.013091e+11,5.506075e+13,0.701179,61.776302,-4.949904,-10.246051,46.580348,9.123636,55.703982,-0.156098,0.857277
std,4.255079e+12,6.223201e+12,2.587640e+13,0.739497,31.800689,7.769972,25.154677,51.802690,10.305236,61.920557,0.369190,0.380820
min,1.001447e+07,1.000661e+06,1.000657e+13,-3.000000,0.000000,-200.000000,-237.500000,-237.500000,-47.500000,-285.000000,-3.000000,0.000000
25%,2.692223e+07,3.295695e+06,3.270317e+13,1.000000,51.670000,-8.340000,0.000000,47.080000,8.375000,56.227500,0.000000,1.000000
50%,4.494514e+07,5.566107e+06,5.522207e+13,1.000000,74.170000,0.000000,0.000000,63.330000,12.660000,76.000000,0.000000,1.000000
75%,7.743106e+07,7.815352e+06,7.736876e+13,1.000000,79.170000,0.000000,0.000000,74.170000,14.840000,89.000000,0.000000,1.000000
max,8.422212e+13,9.977409e+13,9.999554e+13,6.000000,445.000000,0.000000,0.000000,445.000000,63.340000,445.000000,0.000000,6.000000


In [5]:
#data cleaning
#clear all 0 or less ordered products
customer = customer[
    customer["ordered_item_quantity"] > 0]

In [6]:
def encode_column(column):
    if column > 0:
        return 1
    if column <= 0:
        return 0


def aggregate_by_ordered_quantity(dataframe, column_list):


    aggregated_dataframe = (dataframe
                            .groupby(column_list)
                            .ordered_item_quantity.count()
                            .reset_index())

    aggregated_dataframe["products_ordered"] = (aggregated_dataframe
                                                 .ordered_item_quantity
                                                 .apply(encode_column))

    final_dataframe = (aggregated_dataframe
                       .groupby(column_list[0])
                       .products_ordered.sum() # aligned with the added column name
                       .reset_index())

    return final_dataframe

In [7]:
# apply functions to customer
customers = aggregate_by_ordered_quantity(customer, ["customer_id", "product_type"])

In [8]:
customers.head()

,customer_id,products_ordered
0,1000661,1
1,1001914,1
2,1002167,3
3,1002387,1
4,1002419,2


In [9]:
# aggregate data per customer_id and order_id, 
# to see ordered item sum and returned item sum
ordered_sum_by_customer_order = (customer
                                 .groupby(["customer_id", "order_id"])
                                 .ordered_item_quantity.sum()
                                 .reset_index())

returned_sum_by_customer_order = (customer
                                  .groupby(["customer_id", "order_id"])
                                  .returned_item_quantity.sum()
                                  .reset_index())

# merge two dataframes to be able to calculate unit return rate
ordered_returned_sums = pd.merge(ordered_sum_by_customer_order, returned_sum_by_customer_order)

In [10]:
# calculate unit return rate per order and customer
ordered_returned_sums["average_return_rate"] = (-1 * 
                                             ordered_returned_sums["returned_item_quantity"] /
                                             ordered_returned_sums["ordered_item_quantity"])

In [11]:
ordered_returned_sums.head()

,customer_id,order_id,ordered_item_quantity,returned_item_quantity,average_return_rate
0,1000661,99119989117212,3,0,0.0
1,1001914,79758569034715,1,0,0.0
2,1002167,38156088848638,1,0,0.0
3,1002167,57440147820257,1,0,0.0
4,1002167,58825523953710,1,0,0.0


In [12]:
# take average of the unit return rate for all orders of a customer
customer_return_rate = (ordered_returned_sums
                        .groupby("customer_id")
                        .average_return_rate
                        .mean()
                        .reset_index())

In [13]:
return_rates = pd.DataFrame(customer_return_rate["average_return_rate"]
                            .value_counts()
                            .reset_index())

return_rates.rename(columns=
                    {"index": "average return rate",
                     "average_return_rate": "count of unit return rate"},
                    inplace=True)

return_rates.sort_values(by="average return rate")

,average return rate,count of unit return rate
0,0.000000,24823
8,0.013889,1
9,0.066667,1
11,0.083333,1
10,0.125000,1
6,0.166667,2
5,0.200000,2
4,0.250000,5
2,0.333333,13
12,0.400000,1


In [14]:
# add average_return_rate to customers dataframe
customers = pd.merge(customers,
                     customer_return_rate,
                     on="customer_id")

In [15]:
# aggreagate total sales per customer id
customer_total_spending = (customer
                           .groupby("customer_id")
                           .total_sales
                           .sum()
                           .reset_index())

customer_total_spending.rename(columns = {"total_sales" : "total_spending"},
                               inplace = True)

In [16]:
# add total sales to customers dataframe
customers = customers.merge(customer_total_spending, 
                            on="customer_id")

In [17]:
# drop id column since it is not a feature
customers.drop(columns="customer_id",
               inplace=True)

In [18]:
customers.head()

,products_ordered,average_return_rate,total_spending
0,1,0.0,260.0
1,1,0.0,79.2
2,3,0.0,234.2
3,1,0.0,89.0
4,2,0.0,103.0


In [19]:
#instantiate KMeans model
kmeans_model = KMeans(init='k-means++',  max_iter=400, random_state=42)
#fit in the model
kmeans_model.fit(customers.iloc[:,2:])

KMeans(max_iter=400, random_state=42)

In [20]:
# Create the K means model for different values of K
def try_different_clusters(K, data):
       
    cluster_values = list(range(1, K+1))
    inertias=[]
    
    for c in cluster_values:
        model = KMeans(n_clusters = c,init='k-means++',max_iter=400,random_state=42)
        model.fit(data)
        inertias.append(model.inertia_)
    
    return inertias

In [21]:
# Find output for k values between 1 to 12 
outputs = try_different_clusters(12, customers[["products_ordered","average_return_rate","total_spending"]])
outputs = try_different_clusters(12, customers[["products_ordered","average_return_rate","total_spending"]])
distances = pd.DataFrame({"clusters": list(range(1, 13)),"sum of squared distances": outputs})

In [22]:
# Finding optimal number of clusters k
figure = go.Figure()
figure.add_trace(go.Scatter(x=distances["clusters"], y=distances["sum of squared distances"]))

figure.update_layout(xaxis = dict(tick0 = 1,dtick = 1,tickmode = 'linear'),                  
                  xaxis_title="Number of clusters",
                  yaxis_title="Sum of squared distances",
                  title_text="Finding optimal number of clusters using elbow method")
figure.show()

In [23]:
# Re-Train K means model with k=3
kmeans_model_new = KMeans(n_clusters = 4,init='k-means++',max_iter=400,random_state=42)

kmeans_model_new.fit_predict(customers[["products_ordered","average_return_rate","total_spending"]])

array([2, 0, 2, ..., 0, 2, 0])

In [24]:
# Create data arrays
cluster_centers = kmeans_model_new.cluster_centers_
data = np.expm1(cluster_centers)
points = np.append(data, cluster_centers, axis=1)
points


<ipython-input-24-ffdb0ff0bada>:3: RuntimeWarning:

overflow encountered in expm1



array([[2.30640871e+000, 1.36016036e-003, 9.89125990e+042,
        1.19586262e+000, 1.35923618e-003, 9.90002254e+001],
       [3.58638466e+001, 5.90400402e-004, 1.70182308e+233,
        3.60723130e+000, 5.90226185e-004, 5.37034027e+002],
       [8.08634656e+000, 5.40207834e-004, 2.83946777e+109,
        2.20677291e+000, 5.40061974e-004, 2.52025392e+002],
       [2.84522544e+002, 3.31559297e-004,             inf,
        5.65432099e+000, 3.31504344e-004, 1.16004527e+003]])

In [25]:
# Add "clusters" to customer data
points = np.append(points, [[0], [1], [2], [3]], axis=1)
customers["clusters"] = kmeans_model_new.labels_

In [26]:
# visualize clusters
figure = px.scatter_3d(customers,
                    color='clusters',
                    x="products_ordered",
                    y="average_return_rate",
                    z="total_spending",            
                    category_orders = {"clusters": ["0", "1", "2", "3", "4"]}                    
                    )
figure.update_layout()
figure.show()

In [27]:
cardinality_df = pd.DataFrame(
    customers.clusters.value_counts().reset_index())

cardinality_df.rename(columns={"index": "Customer Groups",
                               "clusters": "Customer Group Magnitude"},
                      inplace=True)

In [28]:
fig = px.bar(cardinality_df, x="Customer Groups", 
             y="Customer Group Magnitude",
             color = "Customer Groups",
             category_orders = {"Customer Groups": ["0", "1", "2", "3"]})

fig.update_layout(xaxis = dict(
        tickmode = 'linear',
        tick0 = 1,
        dtick = 1),
                 yaxis = dict(
        tickmode = 'linear',
        tick0 = 1000,
        dtick = 1000))

fig.show()

In [29]:
customers.to_csv("customer_data.csv")

In [30]:

customer_column = [column for column in customers.columns if customers[column].dtype == 'O']
from sklearn.preprocessing import StandardScaler
scaled = StandardScaler
for column in customer_column:
    customers[columns] = scaled.fit_transform(customers[columns])